# MAIN




**Drive secret setup**

In [54]:
import json

with open('/content/drive/My Drive/FYPColab/secrets.json', 'r') as f:
    secrets = json.load(f)



**Libraries download**

In [ ]:
ngrokauth = secrets['NGROK_API']
!pip install ultralytics
!pip install easyocr
!pip install flask-ngrok
!pip install pyngrok
!pip install pymongo
!pip install cloudinary

!ngrok config add-authtoken {ngrokauth}

**Text Sanitization and formatting code**

In [56]:
import re

# Example list of strings to be removed
sanitization_words = [
    "SINDH",
    "PUNJAB",
    "ISLAMABAD",
    "KARACHI",
    "LAHORE",
    "khi",
    "T&C",
    "lhr"

    # Add more strings as needed
]

def text_sanitization(text):
    # Step 1: Convert the input text to all uppercase
    text = text.upper()

    # Step 2: Remove words from the sanitization list (case-insensitive)
    for word in sanitization_words:
        text = re.sub(re.escape(word), "", text, flags=re.IGNORECASE)

    # Step 3: Remove all symbols using regex (keeping only alphanumeric characters)
    text = re.sub(r'[^a-zA-Z0-9]', '', text)

    # Step 4: Remove all spaces
    text = text.replace(" ", "")  # Alternatively: text = re.sub(r'\s+', '', text)

    # Step 5: Move integers to the end of the string
    # Find all integers in the text
    numbers = re.findall(r'\d+', text)
    # Remove all integers from the text
    text = re.sub(r'\d+', '', text)
    # Append numbers to the end of the sanitized text
    text = text + ''.join(numbers) if numbers else text

    return text.strip()



**SMTP and MongoDB**

smtp and mongodb config

In [57]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.utils import formataddr

from pymongo import MongoClient
from datetime import datetime


overspeedingsubject = 'Overspeeding'
warningsubject = 'Unpaid Fine Warning'
sender_email = secrets['SENDER_EMAIL']
sender_name = 'FASTCars Overspeeding Alert System'
password = secrets['EMAIL_PASSWORD']



uri= secrets['DB_URL']
database_name="fypproject"
user_collection="users"
overspeeding_collection="overspeedings"





SMTP

In [58]:

def sendoverspeedingemail( speed, recipient):
    # Create the HTML body with inline styles
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                background-color: #f8f9fa;
                font-family: Arial, sans-serif;
            }}
            .container {{
                max-width: 600px;
                margin: 0 auto;
                padding: 20px;
            }}
            .card {{
                border: 1px solid #dc3545;
                border-radius: 5px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                overflow: hidden;
            }}
            .card-header {{
                background-color: #8B0000;
                color: white;
                text-align: center;
                padding: 10px;
            }}
            .card-body {{
                padding: 20px;
                text-align: center;
            }}
            .card-footer {{
                background-color: #f8f9fa;
                color: #6c757d;
                text-align: center;
                padding: 10px;
                font-size: 12px;
            }}
            .text-bold {{
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="card">
                <div class="card-header">
                    <h3>Overspeeding Alert</h3>
                </div>
                <div class="card-body">
                    <p>You were caught overspeeding with a speed of <span class="text-bold">{speed} km/h</span>.</p>
                    <p>Please adhere to traffic rules to ensure everyone's safety.</p>
                </div>
                <div class="card-footer">
                    This is an automated message from FASTCars Overspeeding Alert System.
                </div>
            </div>
        </div>
    </body>
    </html>
    """

    msg = MIMEMultipart()
    # Set custom display name for the sender
    msg['From'] = formataddr((sender_name, sender_email))
    msg['To'] = ', '.join(recipient)
    msg['Subject'] = overspeedingsubject
    msg.attach(MIMEText(body, 'html'))  # Attach HTML content

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender_email, password)
        smtp_server.sendmail(sender_email, recipient, msg.as_string())
    print('Message sent')

def sendwarningemail(recipient, total_unpaid_fines):
    # Create the HTML body with inline styles
    body = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <style>
            body {{
                background-color: #f8f9fa;
                font-family: Arial, sans-serif;
            }}
            .container {{
                max-width: 600px;
                margin: 0 auto;
                padding: 20px;
            }}
            .card {{
                border: 1px solid #ffc107;
                border-radius: 5px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                overflow: hidden;
            }}
            .card-header {{
                background-color: #ff9800;
                color: white;
                text-align: center;
                padding: 10px;
            }}
            .card-body {{
                padding: 20px;
                text-align: center;
            }}
            .card-footer {{
                background-color: #f8f9fa;
                color: #6c757d;
                text-align: center;
                padding: 10px;
                font-size: 12px;
            }}
            .text-bold {{
                font-weight: bold;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="card">
                <div class="card-header">
                    <h3>Unpaid Fine Warning</h3>
                </div>
                <div class="card-body">
                    <p>Our records indicate that you have <span class="text-bold">{total_unpaid_fines}</span> fines with outstanding amount of <span class="text-bold"> Rs. {2000 * total_unpaid_fines}</span>.</p>
                    <p>Please settle your fines immediately to avoid immediate action.</p>
                </div>
                <div class="card-footer">
                    This is an automated message from FASTCars Overspeeding Alert System.
                </div>
            </div>
        </div>
    </body>
    </html>
    """

    msg = MIMEMultipart()
    # Set custom display name for the sender
    msg['From'] = formataddr((sender_name, sender_email))
    msg['To'] = ', '.join(recipient)
    msg['Subject'] = warningsubject
    msg.attach(MIMEText(body, 'html'))  # Attach HTML content

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender_email, password)
        smtp_server.sendmail(sender_email, recipient, msg.as_string())
    print('Warning email sent')




MongoDB

In [59]:

def find_owner_by_numberplate(db, collection_name, numberplate):
    try:
        collection = db[collection_name]
        # Query to search for the numberplate in the array
        query = {"number_plate": numberplate}
        document = collection.find_one(query)

        if document:
            name = document.get("name")
            email = document.get("email")
            total_unpaid_fines = document.get("total_unpaid_fines") + 1

            update = {"$set": {"total_unpaid_fines": total_unpaid_fines}}
            collection.update_one(query, update)

            if(total_unpaid_fines >= 3):
              sendwarningemail([email],total_unpaid_fines)
            return {"name": name, "email": email}
        else:
            return None
    except Exception as e:
        print("Error while querying the database:", e)
        return None


def insert_into_overspeeding(db, collection_name, data):
    try:
        collection = db[collection_name]
        result = collection.insert_one(data)
        print(f"Data inserted successfully with ID: {result.inserted_id}")
    except Exception as e:
        print("Error while inserting data into Overspeeding collection:", e)


def mongodb_process_overspeeding(numberplate, speed, created_at, number_plate_image):

    # Connect to MongoDB
    client = MongoClient(uri)
    db = client[database_name]

    # Find owner by numberplate
    owner_data = find_owner_by_numberplate(db, user_collection, numberplate)
    name = owner_data["name"] if owner_data else "Anonymous"
    email = owner_data["email"] if owner_data and owner_data["email"] != "Unknown" else 'anonymous@nu.edu.pk'

    # Prepare data for insertion
    overspeeding_data = {
        "name": name,
        "number_plate": numberplate,
        "email": email,
        "overspeeding_amount": int(speed),
        "is_fine_paid": False,
        "created_at": created_at,
        "photo": number_plate_image,
    }

    # Insert data into Overspeeding collection
    insert_into_overspeeding(db, overspeeding_collection, overspeeding_data)

    # Send email if the owner's email is found
    if email and email != 'anonymous@nu.edu.pk':
        sendoverspeedingemail(speed, [email])


#mongodb_process_overspeeding("XYZ555",99.55,datetime.now(),"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxMTEhUTEhMWFhUXFR0YFxcXFxcVFxoXGhcYFxcVFRcYHSggGh0lHRUVITEhJSkrLi4uGB8zODMtNygtLisBCgoKDg0OGhAQGC0dHR0rLS0tLSstLS0tLSstLS0tLSstLS0tLS0tLS0tLS0tLS0tLS0tLS0rLS0tLSstLSstLf/AABEIAK4BIQMBIgACEQEDEQH/xAAcAAABBQEBAQAAAAAAAAAAAAAGAgMEBQcBAAj/xABMEAABAwICBAcIDwcEAwEAAAABAAIDBBEFEgYhMVEHQWFxgZHBEyIycqGxstEUIyQzQlJTYmNzgpKiwtIVFhdEVJOjQ4Oz8GTD4TT/xAAZAQEBAQEBAQAAAAAAAAAAAAABAAIDBAX/xAAmEQEBAAIBBAEEAgMAAAAAAAAAAQIRMQMSIUEEEyJRYYGxIzJx/9oADAMBAAIRAxEAPwCLSQRO108z2H4ruJTmVVVHtyyDk2oVgmY6zi4NAudRsk0+kwAd31xfvSSQbb72XDDq75dup8ezijaPSBh1SNLDyjUqnFNF6WpOeNwDjuSKd8xZmOUste8oyCx35tYSXT0R/mYY3fNlba/WvRzNvNxdIeF6BujmDy67QUVaRttCByhUEONNjPe11O5vLK0J/EMbilaG+yafbc+2s9aCIqPDm5GOIB1BLxGjhyElrdm4KA3SSmEYZ7Jg1C3vrPWoOJ4rBJHlbUwX5ZWDtRZrgzzfKgxGcRnMwalRy1Be4l3GrKXDM383S2+vam2YN/5NJ/fas9Lp9s/61nluq0sVpotGXVLBxXulfsa/8xS/32q30Xo2wzZ5J6fLbUWzMOvmuuumbQnwkVGerI+KLIXbGLbdaMcd0bmmnfI2SnIc649vZe3LdQv3Nqd8H9+P1oqDbWJQaiE6I1A+FBfd7Iiv50r9y6v4sX96L9SPKDxTzDkYXHadivG6FVdxdsdvro/1JGI6H1rjZkIIHGJYrH8adHao0Y11LL71pmlOFROiz6r22oSwPQqsa/NJDlts7+M+ZyvsSoa98RjFO47u+Z+pZsu1sGUdPeZuvVdX2LMySNDXXChs0TxD+lk/D+pSotGsQJGamk/D+pMx3VtcU7tQThenYcEqQ0e0v6l04PU/Iv6l5+y/hvcMF6SZApkeA1B2xuHQpcGBvG2N5+yU9lXdFM252BPMoHHaroYfINkTvuleNNJ8m/7rvUtTAdyDT4a0bVYxxNHEuNgk+I/7p9ScbE74ruorUmmdlhoTjU3kduPUUoA7ikHQltUOWpy8R6ikRYhrGZpbc2BIIv1qOqtAs/4Q8M1ZwEfsKq9JaPPEeZUursMOyFeRF+yTuXl6O6M+UfSCB0JLfgu8E9nOFY6PtZBAa2UB1jaJh2F289PVYlXOMUTZWOY77J3O4ihrSe7KamhPEHE22XAA/MV4ejlMpr29nWll2uMG0dq8VvUVExbDc5dV72OsRM2ADZmPl1q6PBlS/KTdcf6Ea4NE1lFTNbs7jH5WAk9ZJS16tPJsDfwzpvlJ+uP9C9/DKm+Un64/0IrkxVgtYOcHGzSMoDjYnvC4jPqBPe31AqTT1TX6hcEC5aRY24jbjHKNSdQboM/hjTfKTdcf6F7+GFN8rN1x/oR1ddBVqLdAv8Lqb5Wb/H+leHBdTfKzf4/0o8CUE6i3QGOCyn+Vm/x/pXRwV0/ys3+P9KPQUsK1BugD+FNP8tN/j/SuHgng+Xl+6z1I8rKtkTDJI4NY3WXHYNduLlKrBplQ/wBSzqd6lah3QwOCaDjnl+6z1JX8Jqf5ebqj9S0CmqGSNa+Nwc1wuHNNwRvBCbrcRihyCV4ZncGsvfW47ALc4VqDdAf8JIPl5fus9S9/CSH5eT7jVoFbWxwsMkrgxgtdxvYXNhs5TZPRShwDmm4IBB3gi4KdRbrOP4Rxf1Dv7bf1Lh4Io/6l39ofrR7JjdO2XuLpWiQkDIb3u62Ubtdx1qRWYhHFl7o8NzOytvxuOwCytRbrNH8EhB72oaedhafISly8GlVry1PHqBklGroBWppqpqGxsc97srWi7idgHKrUW6y6Pg0rOOpA5pZj+VPt4OKwbK0/3JvUtNp52vY17DdrgHNO8HWD1JuvxGKFodK8MBNgTfWduqw5Fai3Wdt4Pa4bK5/96f1JwaBYhxV8n9+o9S0mnma9rXNILXAEHeCLg9RTdfiMUIDpXhoJsCb7dttStRbrPW6FYkNmISf35j52p1uiWKD+fd/defOxG9HjlNK7JHM1zjsGsE8eq+1WQcnS2zpujGK/17vv387E63R3Fv6934D541oRXlaWwA3AsW/rnfdh7Y06zCMVH82T9iH9KOw5OiQ70IDR4fig/mL/AGIfUpoocQIs9weNxiiI84RgyV28qSyocPhHzoQCLtfc5GhkltRAytdyZbkA8oJHMmahl2kK308jvFn2OaQQePwgD5CVUsfcBx42g9JAJXPONYhv9mDcvIi7kF5cdNgqqFulD+kdD3SIm3fR98Ob4Q6hfoRHEe6NBUmKlFtfTzLyY5dt2+jlj3Y6FOAOzYfSH6CP0AOxVWP4rGwthcSXOyuc0DWY7kEa9WsgNIvsJV1gMGSihZtyNyjmaSB5AEGaT07YppJag5oZmZGn4cbgWuAZqttbmF+Mdf09+Nvl68rF+PRyarSNIvYsyFw4iCHCwPFqukS43CI25Q5hit3M7RYWHc73vraLG/PxBU0Ecfc2hzswA1vLtpsbEx5c7Tx67gcRNrldQ+Bj2mWQhoNy0lr3OA1tGoDK02ub2Jta1rlB0N6SqZK0PjcHNN7EbNRIPlBT4VBoZSvjg74gNc4uiaDfLG45gC7jJuSr9bjBS6CuBdCUUCltKbCWFBT6bNvQVP1d/ukO7FV6LaMUk1FC+SBpc+Pvn6w4nWL3B2q50tZehqvqH+RhPYgvRbQ72TSxy+y5mB2YFjT3oDXuaQNezUeLjKD6XfBLKTRvaTfJO4DdYsjcfxOcelO8I38mf/Kb/wB8ivdH8FjpIRFFci5c5zrZnONgSbADYAOYBUHCk0inhkA97qWuPIMrx58o6U+l7WfCGL0E3Ow9UjFa6POvS05+gj/42od0x0hpJKOZkc8b3OaMrWuBcTmadg1jp2K90Vv7Dpr7e4M9EdllL0B9LcPMldVFhIfHTtmbbbdhjB6cpcRygKx0mxIVFFRz8ZqWZrcTgyTOOtvVZTmNtjTh8akv5QPyoQx9ppe70ZvkEzZ4eQWcLfdcBzxnegth41VaVj3HUfVO8gv2K1AQB+8gdTVkNTLeXNLGwFtja2UAZW28K+1aZFmibr0dP9U0dQsqbSyEVFZS0vEQ57rbrXB/xu61caIj3HT/AFY6iSR5LIdm9kS4lO+lEZdExsftl7AEC9rcebOgrzQWpLqUMd4cL3Ru6DceRwHQmtPcvc4C/wAEVDc3i2dfyXUXRXusNZPDPlD5WibvPBJzG+Xrd90qXwhW9jxkjUJ29WV6vSVkslI+ppRRBuYTAvytLO8BBN8wF9QKPAgLGMUp53QNpme2CdpBEYabbrjXtsehHoTE6ltSLJag61emnawZnua0X2uIaOspTAo2MRuysLQ42kBORokcBlcL5TqO0IR+PEIdZ7rHq1Hv26idl+o9SmiaMAkvaACQdY1EDMQeUDWqWqppAHOaxx9tuAGA5h3BrRcfFJzNNtikewJRZ3cyczi9wFtRAPLtIcR9gLOzpWaZyNdTOcwhzSNRGzwgqSN3etHzG+YK901N6eQlhZfXlO3w9p5Tt6UKzy5JGDidE09IFj5gsZ8N4TdWC8m+6BeXNoAaNV4cACiSWIkaj/8AeZZLhmIGNwWmaPY2HgawbawuGXT1Xvwy7p4FGDFzAY3bHDMOR3H1qVNEHCzgCDtBFx0gqP7JY9t2mx4udLppy7aLEdR5QvR08pPteTrYX/ZHmwmB4s6FhF7kFote1rkbF79lQarxMNiCLtDtY1A6+NTVwru8zwSgk3XQpFroXlxKLCUE2HJwKBNTA2Rjo3i7XtLHDZdrhYi41jUSk4Th8dPG2KEZWNvYXLvCcXHW432kp8JTVIsJurpWSsMcjQ5jtrSLg8Y8qcTZqmD4QUlVFodQg39jsOv4Re4dTiQVftUN2IRN2yNHO4Dzps4zTj/Wj++z1qSSKCLuvdsg7rlyh/wsvxebWUivwiCc5poY5CBYFzQ42vs5tZUZ2kdKP5iL+4z1qzglD2te0hzXC7XA3BHEQRtSjqhT4LTvdnfBE552ucxpJ59WtTQlKDzWrkFMxpJaxrS43cQAC473EbTrO1InqGsFyeK+7UNpJ3Kim02pWm2fNb4rXOHQ7UD0XURJ3FubMWjNawdYXA3A7bbUpzAdTgCOUX6VWYPpDBUnLHIM3xSC13QDt6Lq1KE9FC0aw0DmACdLUN4xpKIJMhYTutbt5VCOm26J33mj8qtoYtSwEFjTIn/SP3x+lXui+MeyDIC3LlaCNd9pI3I2tLxgU2GJRmAbTqA2k8SGsW0kc2QhnVdwsOId6Rr4zzgcSkOmMTyFdDcWfO6QPtqAtYuO3Ne+Zx3BFSxXSA7hFHtL/E7QgbFYrlh1ao223/CujrhG94efmdqzHSbGBBLGNZLoGk7vCeBblRlZryzvV2k+yivIY/eMfFK8uPj8u/1sP0EKjB5GtLjbtUrCYXNppKhrjeNwFt/g+tP1labvZI0tNrWIsU7govQ1bd3feQfpTj5nl0z+27xHPB7C6qgM5eW2eWFtr3IAN7nxtyLcttQQjwKu9xzjdOT1xs9SL37V2xwk4jz59TLLkkoS040jfSxtMYGd7y1ubWAGgFzrcZ1gItKy/hSfrpxySO63NHYVqsRUP04rj/qgc0bO0FFHBzpTPPUmnqHh+djjGcrWuD2Nz5e9AuC0OWbNjJBIBIAuSASAN53BW+htd3GuppOITNB8VxyO8jijbVjcwVDxR5DNRt4XWGOI8oCnzR5XObucR1FQq8am3+MB1gjtWmGJDH6si5qp9nFK8eYreqR2anpn7TJTRvcd7iCCTz2XzpGNi+g9HnZsPoD/AOK0dTnBEaqa1LCQEsLTBQWI6dN91m/xG7ekecFbcsZ4QWWq/sAdUkiKcQ00DcE60o/4L8Ep6iKodLFHI5j2AZ2h1muDtl9msI1j0apBspqcf7EX6UaNrDmvG8LdNB33wyk5pB/lcpEWEQjZHGOaOMeZqnxtsABsGzk5kyC04F1JCarnWjdbdbr1dqWWb6c4y6SQwtPeixfyk981p5AC0855AhuMrmIVGeaV2+Rx6Mxt5LLS9GtEYRTRumja+SRoe7MA7KHa2tANwLC1+W6GuGeQSFpBaSCDcEGxBGsEHiWx6OYl7JpWTHw75JLfHbx9IselZ5plgrKd4dGLNcbEDYDa7SBxXAdq2d7q3Ig4Lqi8dRHuLHjnN2nyAKRek2DSzTAsAtym27Zv41Xs0Vn+Z+L9K0ILqRtnFfgkkLMznMIuBYF19fHraNSveDuT22UHji/MPWoOlmJZ39zadQsT+UeUu6W7lY6B0pAfLxEZG8o2k+Yday16XmkGKCNhDdfa7aG8w2noHGgfMTrNyb3unMZe7urhc6reYH1qf7C7nSF7h3z7EX+C0EeU36rJC74PHe2yeIPOj4FZvoHKe7P8T8wR9HKs2GXQY4SD7nk+rKAMVw5kpjc4XPcwOgOd60e8In/5pT9E7zFCD/Bj5WfmcOxYzn2meaof2NH8VeV5cLy49rep+FFimGRTstK0Hc7jHMUOYZhL4YqyNwOV0V2HeAHjr2K0gxPO/Ue9HlKuq9ocyw44X9nrWOlbLp7OtjO3aBwIS+1VLdz2Hra4flR1INazrgNd39UN7Yz1F/rWjzDWV7Zw+feTLlkvCY+88Td0IP3nv9S1mbYeZY9wivvWW+LFGPIXfmRTis+Dah7pBWtOx7Gx9JbJ+oIFFxyEeQhahwVR2pKmTjEot0Nb6ygDSOm7lVTs4hK4jxXHM3yOCj7b57JErYphsmhjk+8wX/FmUauGocjm+kFXaB1PdcLpnbTGZIT9l+Zn4XjqVjX+AeSx6iCn0zeWCVTbSPG57h1OIW6aHSXwyi5GPb1P/wDqxLGm2qZxumk/5HLY+Dx98Kp/myzD8YsqHLgQhKCQ1ONWmHVkHCWy1U3xT6bj+Za+Asp4UG+3s+35oz2opi44FXd7XN+ZC78bwexaIsG0f0hnoy90BaO6NDXBzS4EA3GwjWDfrVm/T2uPw2DmjHbdUps22hKCz/g10hqKieaOeTOO4l7RlY3KWvYNRaAdjuO60ABLNmnQma/wDzj0gn0zWC8bvFJ6RrUGETNsXc57V9AxtAawDZkbbmyhYNiUWWaVu6Rw6Mxstl0cxeOoponNcC8Rhsjb3c17RlOrbY2uDuRGqoOEhveNPzm+aVRuC53t0w3w+Zw/70p/hIf3jB85vkbIe0dai8F49vmO6HzuHqUvTRAq/HsQEMTid2zfxW6TYdJVgECafTO7oxtzlJOrlAbb0ndaRFNTRunlAGt8jvKdZPMNZ5gtTo6cRsaxuxosO0rMdHsW9jSd0yB5tYAki17axbjWiYJi0dTGXs71zSA9h4r7CDxj1INdfg0RdmLbnlsehRtLdUHN62q5BVPpWLwHm7QkKrQWT2931Z9JqPGyrPdCTaoNvk3edqO2lBqj0+dekm+qd6JQrN4EX1f/ALHon07PuKf6p/olCrj7XD9Wf+R659ThrE2vLy4uOnRj9LXuYRYo70ex5lRlj+G1jrjku3Wszyoj0CdarA3xuHmPYt9k3trvutCHgUflqp2fQ3+7I0fmWo1A74rK+CvvcVmb8yUdUjT2LVqod8V2nDjeUOpPeu5isV05devn5HNHVGwdi2qqHeno86wvSaXNWVJ+neOgOLR5AEVRp3BbDbDnfPmd1AW7EDcIlNlq8/ykTSfGbeM+g3rWi8HUWXDIfnPefxut5EK8KtLqik3Pew/aAePRcqqcrTgdqs1NVw/EkjmHM4GN1upqL64e1u8U+ZZpwPVeWudFfVNTyMtvc0CRvoFadUC7HeKfMmcC8sO0obasqPrXeU37VqfBi6+GN+bUSDr1rMdMG+7Z/GaeuNh7VovBM69BKPi1WrmMTD57qhvAxalBJCUAtMFhZfwrM9tiP1now+orTwVnXCs33s/OHla79ARTiFNF8AdWyuiY8MysLySC7UC0WABHxkVw8GR+FUk+LDbymRQeCR3u14300nnYtZaqK0M6I6IMo5XyiR7y6Mx2cGgAEtcTYXJPeBFYSAuhIOBcI1LwXlBkGnFA6KpLvgvHlbZpHVld9pU9HA6R7WMF3uIa3nOrqWy41gcdS3LI2/HuNxxgjWDt82tVOH6O09CTUd84tae+e4HK3KS4tAaBewtfl5Uaa2HNP6oCSKmabinia085a2w6Gtb95XfBdB3k8u8tYOgFx9Jqz2qq3SyPkdte4uPSb26Ni2TRbD+4UsUZHfZcz/Gf3xHRcDoUquFUYtgMc7g599W423DsHUpFZi8ERtJK1h+c4A9G/b5VIoayOVgfE4Pbci43jaEgH6Q6OxxRF8dwRy3B169vPfoTWgU5FSW8To3DqsR5kS6VD3NJ4rvQcg7Qh/uyMbw4fgPqQWmAqq0o94fzHzK0Cq9Jh7nf4rvRKgG9CZPdP2Hdi0BpWb6EO91N8V3mWjgKNUWnZ9w1H1L/AEShSM3gpz9Gf+R6LNOR7hqPqX+iUH0jvc1N9Wf+R6xnwcTi4uXXlxdWHq70Mdasi5cw/AfUqRWmi77VcPj26wR2rrQKNBTkxtzd7ph1hzuxa5VbVj+BnLjzOWU/iiPrWwVY1rePDGSFUcXjDzr5+r5c0sjvjSOPW4lb7XvytvuueppPYvnsFFWLetEY8uHUg+iB+8Ae1UfCVTh1JI74jo3dOfufmkWbwaS1jI2xsqJGsaLNAI1DiANrqNVYpPKLSTyvG58j3DVs1E2VtaTtDq7uFdSy8TZ2X8VxyO8jit1qo7FzdxIXzkLjWNo1jn4l9Fx1IljimGyWJj/vNF/LdMWTFNN2WrZOVsZ/xM9SOeCF3uSpG6dp62AdiDeEFtqvniZ5Mzfyor4Hn+1VbfnRnrzDsCpyrwPgo2IYnFA3NK8MG86huUkIO4TW+5ieVvp27UsReYXpTSTzMgimBkebNGWQAkAnwi22wHj4kMcKrO8jP0jR+Cb1IT0FdbEKY/S+drh2o24V4/ar7pB/7G/mU17D3BQ+2IN5YpB+EHsWvBYvwZyBuIw3NgQ9uvljctilqmN8J7BzuAHlVBkkLyqpdJKRmp1TCDu7o0nqBSaDSijlmZBHO0yPNm2a+xNidbsths3pGlyEoJK6FAtqBuEzFMsYhB1vNj4rbOd1nIPvI2kfYE7liWlmI93qXuvdrTkb0E5j0uLjzEIpkStC8P8AZFXGwjvWnO/xWa/Kco6VtZKA+CrDssMlQRrkdkb4jNvW64+yjtUVZxwjMtLGeI5h6B7VbcGuJx9xkhc9rXiTO0ONswIAIbvII8qe0/wsyQ5mC7mHNYbTYEOA6Df7KzBr1HlqWmmMxiIxtc0vcLWBvqO0m2zVfrHKqDQNmarafisc7yZfzISa9aXwfYO6NjppBZ0lsoO0M23O654twG9SXmJ4/DAcshIJ2WBO7dzqixfS2CSJ7GCQktIFwALkEa9d+PcqvhGNpY+Z35PWhqnaXeCCeYE+ZSkWmCYp7HmEuXNYEWvbaCNqPdGdI/ZTnt7mGZQCO+vtJBvqCzyLDJ3bIZD9h3qRfoJhssT5HSMLQ5oAuRe977LoVXGmg9xVH1L/AECgjCtdHSnex3plHOl4vRVA+hf6BWcUGKMZQ0mbV3jzY7ffCOxZz4axWi8qj94Id66uHl1ZOpmCPtUwH6VnpAKESp2CU5dK13EHA35iCu2V1N1mTYpvkxyE75Y/K0NWx1m1YxpAcuK07vnwnqkt2LaKzat4sZKDSabLTyu+LDIfwH1rCI2EkAaybADeTqAW0afTZaOc/R5fvvY3tWSYDFnqYG75mD8YuinEXxcGE2rPUMaeMBjnW6yLqZFwZMAvJUvO/LG1vncVpFR4R5029twRvFk6Z3Xz/idGYZpIibmN5bffY2B6RY9K2Tg/qu6YbTnjjL4jzNcS0fdcFmfCBBlrXn5RjH/hDT+JjkYcD1VeCph+LIyQfaGQ+gicm8BvhKZaqZywjySzDsCj6H6VuoTLaISCVoFi7JYtJIN7HerHhTZaaE/MeOqS/wCZBQcBtKl6H8nCZMfBp4xzvc7zAKmxrS2oqmGOQRhpIJyNcDqNxrc48dkPwsc7wWl3igu8ysYMDqn+DTTH/aePOErUSNE5MtbTH6Zo6zbtWjcKjfaHnc4f8jR2oY0O0OqPZMck8ZijjcH99bM4tN2gNBvtANzuRvpthT6qB0cZAc61i6+UEPY/WWgkDvCNnGn0LfLFmldDRuCN6bgzm+FURjxWPf5y1WUHBmz4c8h8VjW+kSrS3Gcq50MfavpT9M3y6u1HcHBxSjwjK7ne0eixWuGaG0kMjZWRd+03aS+R1jvsXWv0K0NiV+085866Emy6EsqjSqaRtM8wgl+U5QBc3sbWHGRttyLJqfReseQBTSC+q7mlgHKS62pbiWX2i6UyAfFCjLoxhdE2GGOFuyNgaDvIGsnnNypi6GJiariZ4crG872jzlSOvjDhYoaxDQinkcXZS0k3OQ5b8pFiOoBWztIaMfzEZ8U5vRuo0mmFE3/VJ+y4ekAheTWF6H00Lg4R5nDY55zkcoGpt+W10RtahOfhGoWfC63MHmcSo54TKc+9Rl/iiR/osVtaovfRtcbkAnmBTraccvWUGDTiod73Rzn/AGHN8r3NSHaS4m46qVzBvfLAzyXcVbWh02k+Z5E82mI1WA6QFnL8XxA+FJSs8aoe49UcYv1qO+pqT4dfCORkEsnlfJZFyh7RjptOyOkmzuAvG5ttpJIsAN+3iWaT4K10UMbtRjiDTyOJL3DoLrdCs32zBzpJJnjW10mVrWkfCbGwWvuLibcWteBXHPP1HTGaUH7ss3ryILry57bYpa5srHuxYBGzp5ypOFYd3hmeNQ2cpUzAsPDs08g71uxGfVx8++3+zMdfyl6ed5WQSfMafuvJW11fEsW04Bmp6apbssWO5Cdnla4dS1vBsRZVUsU8ZvdgzDja4CzmnlBuvVi45KHTjDZKimfHELvOUgXAvleCRc6tmvoQhofoVVx1cMs0bWMY/MbvY46gbWDSeOy1IhdAWtDbrzck8q7ZJLgNptz6lFnxmmj8OeJvPIwdqQp8c0NhqpGvlMgLQWjI4C7cxcAbg7C52zerDR3R2Cja8Qtdd9sznOzEgXsNwGs8SjVGm1AzbUsPih7/AEAVAn4SqFvgmV/ix29IhHheV/W4LBMQZYmSWJIztDrE2va++wTlNg8DPAhiZ4sbG+YILn4V4B4FPIfGcxvmuoJ4VZnaoqRt/Hc/yNaFbi1Wnti/7s8yX3JZk3TPF5Pe6IAfUynyucAvPxTH5NTY8nMyFvlkJVtdrUGxpeS23UsmlosXcPb66OIEWIdUZLX5Iha6rpNH43e/YtG7fbNN+dXcu1sE+IwR65Jom+NIwddyoEul1A0a6qM+KTJ6AKyY4bhbPCq6h9vk4snpNUqjiws7IKl/jyBvoFFzkPa0CXhFw9uyR7uaNw9MNVdUcK1IPBilPjFjfM4odhkom+Bh8f8AuPdJ6V1YUuJhvvVPTx+LEAud62M9tTp0+eFRz/eaMu+09/kaztSf31xWT3qiIHLC/wA73AeRTI8XnPw7cwaOxK9lSHa9x6Sud+Ti19Kq84jj0h8HuY56do7XJElDi7/DrWM/35PRY0AqzIJ2m6l0sKxflfo/TUQ0OqZPfK+/Mxz/ACvepEXB7GPCqZz4vc2flKLaaNPyBV+RlpdkBWIaL0cLC9zJpbcTp3j0bIOk0jpWXDMKiuOOSR8vpDtWmYm6+pBuK4UwG+ULWHUy1ui4zelCNO5m+801JEOLJAAfP2JDtPsQcdc5aNzGRtHo38qn/s+M/BC5+yo/irf1F2pNFiE89i6eU/bdbqvZXT33ABN7Dj1qqpKdrNinMcudpSGFPNKYaU61ZJ9qWE00pxpUi15cuvKQjj4PqV1CROXMIBLbHLaw1auO6zXSothYynjGr4S1bSXSRk4DY2uAYMxzW1k7BqJ1LEsWqTJK5x3rzfH6f+Ts9Yc/u1rPLxv8pWA4pG1jqeoGaCTb807xbXbZs1gi6mQ6KVUR7phtXdjteqQsPICR3rumyG3BKgqHsN2Pc0/NcW+ZfUcRN3LHzqzO589N67rr9HcZf75VZeedzR/jFkOS4vUEW9kTD/cf61GdK9w797n+M4u86kvZ9C3nXU4hADx5nuefxEJA0aw9nvleXH6KInqtmVGw2SXvQF8KfCGf1UvU0flK6/E8Nj1x4eXn6SVx8hLgh0jVdepYs7hr40W6MgwbpCxkJkjoaaM/B7zNr37BdVE+n9bazHRx+JE3VyDNdQsXnNg0bAFQuOtZwuWvLVkXM+l1c/wqqX7JyeiAorcUlfqfLI7xnud5yq8rwK1fKSjHd2xWGwWUGFyl5rhc6kaZ1zZX2HQ2aFQM8NE9GdQWOrdNYxMp4la08NlGpGq1iavJlk6yORMKmwwr0LFKYsdyKhhVhTQ6wFEjepsEllTLyqtpacNaFAndYJTpSVCqpL6l2n35ajnxFbUG5UKpizBTZwoxK9eteHMM1MOUpDVcYpCLXVMCud8NynWOT8ZUdpTrUJLaU80qKxSGFSPNKcaUy0pxpSjq6kZl5Sf/2Q==")

**Using API with video processing**

imports


In [ ]:
from flask import Flask, request, jsonify
import cv2
import easyocr
from ultralytics import YOLO
import os
from pyngrok import ngrok
import tempfile
import base64
import cloudinary.uploader
import cloudinary.api

cloud_name =  secrets['CLOUDINARY_CLOUD_NAME']
api_key = secrets['CLOUDINARY_API']
api_secret = secrets['CLOUDINARY_SECRET']

cloudinary.config(
    cloud_name = cloud_name,
    api_key = api_key,
    api_secret = api_secret,
    secure=True
)




video processing function


In [61]:
def process_video_stream(cap, speed, timestamp):
    detections_list = []  # List to store number plate detections

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break  # Exit the loop if no frames are left

        # Run YOLO detection on the frame
        results = model.predict(source=frame, conf=0.4)

        for detection in results[0].boxes:
            confidence = detection.conf.item()  # Extract confidence
            class_id = int(detection.cls.item())  # Get the class ID
            bbox = detection.xyxy[0].cpu().numpy()  # Extract bounding box

            # Check if the class ID corresponds to a number plate
            if confidence > 0.5 and class_id == 0:  # Assuming '0' is the class ID for number plates
                detections_list.append((confidence, bbox, frame.copy()))

    cap.release()  # Release the video capture object

    # Process the detections for EasyOCR
    highest_ocr_result = None
    highest_confidence = 0
    highest_cropped_frame = None
    highest_text = ""

    for yolo_confidence, bbox, frame in detections_list:
        x1, y1, x2, y2 = map(int, bbox)  # Convert coordinates to integers
        cropped_frame = frame[y1:y2, x1:x2]  # Crop the frame

        # Use EasyOCR to read text from the cropped frame
        result = reader.readtext(cropped_frame)

        # Calculate the average OCR confidence for the cropped frame
        avg_ocr_confidence = sum(detection[2] for detection in result) / len(result) if result else 0

        # Update the highest result if this one has a higher OCR confidence
        if avg_ocr_confidence > highest_confidence:
            highest_ocr_result = result
            highest_confidence = avg_ocr_confidence
            highest_text = ' '.join([detection[1] for detection in result])
            highest_cropped_frame = cropped_frame  # Save the cropped frame

    # If no OCR results are found, return an error message
    if not highest_ocr_result:
        return jsonify({"error": "No number plate detected with OCR results"}), 400

    # Upload the highest-confidence cropped frame to Cloudinary
    highest_cropped_image_url = None
    if highest_cropped_frame is not None:
        _, buffer = cv2.imencode('.jpg', highest_cropped_frame)
        upload_result = cloudinary.uploader.upload(buffer.tobytes(), resource_type="image")
        highest_cropped_image_url = upload_result['secure_url']

    # Find the next available file ID (check the existing files in the folder)
    existing_files = os.listdir(ocr_results_folder)
    existing_file_ids = []

    for f in existing_files:
        if f.startswith('ocr_results_') and f.endswith('.json'):
            try:
                file_id = int(f.split('_')[2].split('.')[0])  # Extract the number after 'ocr_results_'
                existing_file_ids.append(file_id)
            except ValueError:
                continue

    # Get the next file ID
    next_file_id = max(existing_file_ids, default=0) + 1  # Increment to the next file ID

    # Save the OCR result to a JSON file
    file_name = f"ocr_results_{next_file_id}.json"
    file_path = os.path.join(ocr_results_folder, file_name)

    numberplate = text_sanitization(highest_text)

    # Prepare the JSON structure
    ocr_result_data = {
        "numberplate": numberplate,
        "timestamp": timestamp,
        "speed": speed,
        "number_plate_image": highest_cropped_image_url
    }

    # Write the JSON data to the file
    with open(file_path, 'w') as json_file:
        json.dump(ocr_result_data, json_file, indent=4)

    mongodb_process_overspeeding(numberplate, speed, timestamp, highest_cropped_image_url)
    # Return the file ID in the response
    return jsonify({"status": "success", "file_id": next_file_id}), 200

In [ ]:
# Initialize Flask app
app = Flask(__name__)

# Setup Ngrok tunnel to expose the Flask app to the public
public_url = ngrok.connect(9080)
print(" * ngrok tunnel \"{}\" -> ".format(public_url))  # Print the Ngrok URL


# Enable Flask debugging
app.config['DEBUG'] = True  # Set debug mode here

ocr_results_folder = "/content/drive/MyDrive/FYPColab/ocr_results/"

# Initialize EasyOCR reader and YOLO model
reader = easyocr.Reader(['en'])  # English language for text recognition
model = YOLO("/content/drive/MyDrive/FYPColab/best.pt")

@app.route('/process', methods=['POST'])
def receive_data():
    # Check if a file is part of the request
    if 'video' not in request.files:
        return jsonify({"error": "No video file uploaded"}), 400

    # Get the JSON data from the form
    json_data_str = request.form.get('json_data')  # Get the JSON string from the form
    speed = None
    timestamp = None

    if json_data_str:
        try:
            json_data = json.loads(json_data_str)  # Parse the string into a dictionary
            print("Received JSON body:", json_data)

            # Extract speed and timestamp values
            speed = json_data.get("speed", None)
            timestamp = json_data.get("timestamp", None)

            # Validate speed and timestamp
            if speed is None or timestamp is None:
                return jsonify({"error": "Missing 'speed' or 'timestamp' in JSON data"}), 400
        except json.JSONDecodeError:
            return jsonify({"error": "Invalid JSON data"}), 400
    else:
        print("No JSON body found.")
        return jsonify({"error": "No JSON data provided"}), 400

    video_file = request.files['video']

    # Save the uploaded video to a temporary file
    with tempfile.NamedTemporaryFile(delete=True) as temp_video:
        video_file.save(temp_video.name)  # Save the file to the temporary location
        cap = cv2.VideoCapture(temp_video.name)  # Open the video file

        if not cap.isOpened():
            return jsonify({"error": "Failed to open video file"}), 400

        # Process the video stream and pass the speed and timestamp
        return process_video_stream(cap, speed, timestamp)

@app.route('/', methods=['GET'])
def hello_world():
    return "Hello, World!"

if __name__ == '__main__':
    # Run the Flask app
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=9080)


# Code for training

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Continue with your training script
from ultralytics import YOLO

!yolo task=detect mode=train model='yolov8n.pt' data='D:/FYPProject/data.yaml' epochs=30 imgsz=640 batch=4 name='yolov8n_custom'

In [ ]:
!pip install roboflow
import json

with open('/content/drive/My Drive/FYPColab/secrets.json', 'r') as f:
    secrets = json.load(f)

from roboflow import Roboflow
roboflow_api = secrets['ROBOFLOW_API']
rf = Roboflow(api_key= roboflow_api)
project = rf.workspace("roboflow-universe-projects").project("license-plate-recognition-rxg4e")
version = project.version(4)
dataset = version.download("yolov8")


In [ ]:
import ultralytics

!yolo task=detect mode=train model=yolov8s.pt data='/content/License-Plate-Recognition-4/data.yaml' epochs=15 imgsz=640 batch=64 name='yolov8s_epochs15batch64'


# Code for testing

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO('/content/best.pt')  # Adjust path as needed

# 3. Define the test data path
test_data_path = '/content/test'  # Replace with your test data path

# 4. Run inference on the test data
results = model.predict(source=test_data_path, save=True, conf=0.5, imgsz=640)

# 5. Process the results
for result in results:
    df = result.pandas().xyxy[0]
    print(df)  # Print predictions for each image
    result.show()  # Display the image with predictions

# The results will be saved to the default directory (runs/predict)

In [ ]:
!yolo task=detect mode=val model=/content/runs/detect/yolov8s_epochs15batch64/weights/best.pt data='/content/License-Plate-Recognition-4/data.yaml' name='validation_test'

In [ ]:
!yolo task=detect mode=predict model=/content/best.pt conf=0.45 source='/content/video4.mp4'

In [ ]:
from google.colab import files

files.download('/content/yolov8s.pt')

# Multiple easyocr test codes

In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
from google.colab.patches import cv2_imshow  # Use this in Colab for displaying images

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # English language for text recognition

# Load the YOLO model
model = YOLO("/content/best.pt")

# Path to the video
video_path = "/content/video1.mp4"

# Open video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of highest confidence detections
detections_list = []  # List to store detections

# Read frames from video
frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frames are left

    # Run YOLO detection on the frame
    results = model.predict(source=frame, conf=0.5)

    # Access detections directly from the results
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Extract confidence
        if confidence > 0.5:  # Set minimum confidence threshold
            detections_list.append((confidence, frame.copy(), detection))

    frame_id += 1

# Release the video capture object
cap.release()

# Sort detections based on confidence values in descending order
detections_list.sort(key=lambda x: x[0], reverse=True)

# Take the top 10 detections
top_detections = detections_list[:10]

# Process the top detections
for confidence, frame, detection in top_detections:
    # Get the bounding box for the detection
    bbox = detection.xywh[0].cpu().numpy()  # Get bounding box as [x_center, y_center, width, height]

    # Extract bounding box values
    x_center, y_center, width, height = bbox
    x1 = int(x_center - (width / 2))
    y1 = int(y_center - (height / 2))
    x2 = int(x_center + (width / 2))
    y2 = int(y_center + (height / 2))

    # Crop the image based on the bounding box
    cropped_image = frame[y1:y2, x1:x2]

    # Display the cropped image
    cv2_imshow(cropped_image)

    # Use EasyOCR to read text from the cropped image
    result = reader.readtext(cropped_image)

    # Print the confidence value and detected text
    print(f"Highest Confidence Value: {confidence}")
    if result:
        for detection in result:
            print(f"Detected text: {detection[1]}")
    else:
        print("No text detected in the image.")

if not detections_list:
    print("No detections were made with confidence above the threshold.")


In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
from google.colab.patches import cv2_imshow  # Use this in Colab for displaying images

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # English language for text recognition

# Load the YOLO model
model = YOLO("/content/best.pt")

# Path to the video
video_path = "/content/video2.mp4"

# Open video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of detections
detections_list = []  # List to store number plate detections

# Read frames from video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frames are left

    # Run YOLO detection on the frame
    results = model.predict(source=frame, conf=0.5)

    # Access detections directly from the results
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Extract confidence
        class_id = int(detection.cls.item())  # Get the class ID

        # Check if the class ID corresponds to a number plate
        if confidence > 0.5 and class_id == 0:  # Assuming '0' is the class ID for number plates
            bbox = detection.xywh[0].cpu().numpy()  # Get bounding box as [x_center, y_center, width, height]
            detections_list.append((confidence, frame.copy(), bbox))

# Release the video capture object
cap.release()

# Process the detections for EasyOCR
ocr_results = {}

for confidence, frame, bbox in detections_list:
    # Extract bounding box values
    x_center, y_center, width, height = bbox
    x1 = int(x_center - (width / 2))
    y1 = int(y_center - (height / 2))
    x2 = int(x_center + (width / 2))
    y2 = int(y_center + (height / 2))

    # Crop the image based on the bounding box
    cropped_image = frame[y1:y2, x1:x2]

    # Use EasyOCR to read text from the cropped image
    result = reader.readtext(cropped_image)

    # Store the detected text and confidence in a dictionary (to avoid duplicates)
    for detection in result:
        text = detection[1]
        ocr_confidence = detection[2]  # Confidence of the OCR result

        if text in ocr_results:
            ocr_results[text].append(ocr_confidence)  # Append confidence if text already exists
        else:
            ocr_results[text] = [ocr_confidence]  # Create a new entry

    # Display the cropped image with detected text
    cv2_imshow(cropped_image)

# Prepare a list to store average confidence for each detected text
average_results = []

for text, confidences in ocr_results.items():
    average_confidence = sum(confidences) / len(confidences)  # Calculate average confidence
    average_results.append((text, average_confidence))

# Sort results by average OCR confidence and take the top 5 matches
average_results.sort(key=lambda x: x[1], reverse=True)  # Sort by average confidence
top_5_results = average_results[:5]

# Display the top 5 results
for idx, (text, avg_conf) in enumerate(top_5_results):
    print(f"Result {idx + 1}: Detected text: '{text}' with Average OCR confidence: {avg_conf:.2f}")


In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
from google.colab.patches import cv2_imshow  # Use this in Colab for displaying images

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # English language for text recognition

# Load the YOLO model
model = YOLO("/content/best.pt")

# Path to the video
video_path = "/content/video4.mp4"

# Open video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of detections
detections_list = []  # List to store number plate detections

# Read frames from video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frames are left

    # Run YOLO detection on the frame
    results = model.predict(source=frame, conf=0.4)

    # Access detections directly from the results
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Extract confidence
        class_id = int(detection.cls.item())  # Get the class ID

        # Check if the class ID corresponds to a number plate
        if confidence > 0.5 and class_id == 0:  # Assuming '0' is the class ID for number plates
            detections_list.append((confidence, frame.copy()))  # Store YOLO confidence and the frame

# Release the video capture object
cap.release()

# Sort detections by YOLO confidence and keep the top 5 frames
top_5_detections = sorted(detections_list, key=lambda x: x[0], reverse=True)[:5]

# Process the top 5 frames for EasyOCR
ocr_results = []

for yolo_confidence, frame in top_5_detections:
    # Use EasyOCR to read text from the entire frame
    result = reader.readtext(frame)

    # Concatenate all detected texts in the current frame
    detected_texts = []
    for detection in result:
        text = detection[1]
        ocr_confidence = detection[2]  # Confidence of the OCR result
        detected_texts.append((text, ocr_confidence))  # Store text and its confidence

    # Only add if there are detected texts
    if detected_texts:
        combined_text = ' '.join(text for text, _ in detected_texts)  # Combine texts into a single string
        avg_ocr_confidence = sum(conf for _, conf in detected_texts) / len(detected_texts)  # Average OCR confidence
        ocr_results.append((combined_text, avg_ocr_confidence, yolo_confidence, frame))  # Store results

# Sort results by EasyOCR confidence
ocr_results.sort(key=lambda x: x[1], reverse=True)  # Sort by EasyOCR confidence

# Display the results
for idx, (combined_text, easyocr_conf, yolo_conf, frame) in enumerate(ocr_results):
    print(f"Result {idx + 1}: Detected texts: '{combined_text}'")
    print(f" - Average EasyOCR Confidence: {easyocr_conf:.2f}")
    print(f" - YOLO Confidence: {yolo_conf:.2f}")

    # Show the specific frame
    cv2_imshow(frame)


In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
from google.colab.patches import cv2_imshow  # Use this in Colab for displaying images

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # English language for text recognition

# Load the YOLO model
model = YOLO("/content/best.pt")

# Path to the video
video_path = "/content/output_5.avi"

# Open video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of detections
detections_list = []  # List to store number plate detections

# Read frames from video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frames are left

    # Run YOLO detection on the frame
    results = model.predict(source=frame, conf=0.4)

    # Access detections directly from the results
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Extract confidence
        class_id = int(detection.cls.item())  # Get the class ID
        bbox = detection.xyxy[0].cpu().numpy()  # Extract bounding box

        # Check if the class ID corresponds to a number plate
        if confidence > 0.5 and class_id == 0:  # Assuming '0' is the class ID for number plates
            detections_list.append((confidence, bbox, frame.copy()))  # Store YOLO confidence, bounding box, and the frame

# Release the video capture object
cap.release()

# Print total YOLO detections found
print(f"Total YOLO detections found: {len(detections_list)}")

# Sort detections by YOLO confidence and keep the top 5 frames
top_5_detections = sorted(detections_list, key=lambda x: x[0], reverse=True)[:5]

# Process the top 5 frames for EasyOCR
ocr_results = []

for yolo_confidence, bbox, frame in top_5_detections:
    # Extract the bounding box coordinates (ensure it's a list)
    x1, y1, x2, y2 = map(int, bbox)  # Convert coordinates to integers

    # Crop the frame to the bounding box area
    cropped_frame = frame[y1:y2, x1:x2]

    # Use EasyOCR to read text from the cropped frame
    result = reader.readtext(cropped_frame)

    # Print the number of detected texts
    print(f"Detected texts in frame: {len(result)}")

    # Concatenate all detected texts in the current frame
    detected_texts = []
    for detection in result:
        text = detection[1]
        ocr_confidence = detection[2]  # Confidence of the OCR result
        detected_texts.append((text, ocr_confidence))  # Store text and its confidence

    # Check if any detected texts are present
    if detected_texts:
        # Combine texts into a single string, filtering out empty strings
        combined_text = ' '.join(text for text, _ in detected_texts if text.strip())

        # Calculate average OCR confidence
        avg_ocr_confidence = sum(conf for _, conf in detected_texts) / len(detected_texts)

        # Store the combined text, average OCR confidence, YOLO confidence, and the cropped frame
        ocr_results.append((combined_text, avg_ocr_confidence, yolo_confidence, cropped_frame))

# Sort results by EasyOCR confidence
ocr_results.sort(key=lambda x: x[1], reverse=True)  # Sort by EasyOCR confidence

# Display the results
for idx, (combined_text, easyocr_conf, yolo_conf, cropped_frame) in enumerate(ocr_results):
    print(f"Result {idx + 1}: Detected texts: '{combined_text}'")
    print(f" - Average EasyOCR Confidence: {easyocr_conf:.2f}")
    print(f" - YOLO Confidence: {yolo_conf:.2f}")

    # Show the cropped frame
    cv2_imshow(cropped_frame)


# Video processing and EasyOCR with 5 outputs

In [ ]:
from ultralytics import YOLO
import cv2
import easyocr
from google.colab.patches import cv2_imshow  # Use this in Colab for displaying images

# Initialize the EasyOCR reader
reader = easyocr.Reader(['en'])  # English language for text recognition

# Load the YOLO model
model = YOLO("/content/best.pt")

# Path to the video
video_path = "/content/video3.mp4"

# Open video file
cap = cv2.VideoCapture(video_path)

# Variables to keep track of detections
detections_list = []  # List to store number plate detections

# Read frames from video
while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break  # Exit the loop if no frames are left

    # Run YOLO detection on the frame
    results = model.predict(source=frame, conf=0.4)

    # Access detections directly from the results
    for detection in results[0].boxes:
        confidence = detection.conf.item()  # Extract confidence
        class_id = int(detection.cls.item())  # Get the class ID
        bbox = detection.xyxy[0].cpu().numpy()  # Extract bounding box

        # Check if the class ID corresponds to a number plate
        if confidence > 0.5 and class_id == 0:  # Assuming '0' is the class ID for number plates
            detections_list.append((confidence, bbox, frame.copy()))  # Store YOLO confidence, bounding box, and the frame

# Release the video capture object
cap.release()

# Print total YOLO detections found
print(f"Total YOLO detections found: {len(detections_list)}")

# Sort detections by YOLO confidence and keep the top 5 frames
top_5_detections = sorted(detections_list, key=lambda x: x[0], reverse=True)[:5]

# Process the top 5 frames for EasyOCR
ocr_results = []

for yolo_confidence, bbox, frame in top_5_detections:
    # Extract the bounding box coordinates (ensure it's a list)
    x1, y1, x2, y2 = map(int, bbox)  # Convert coordinates to integers

    # Crop the frame to the bounding box area
    cropped_frame = frame[y1:y2, x1:x2]

    # Use EasyOCR to read text from the cropped frame
    result = reader.readtext(cropped_frame)

    # Print the number of detected texts
    print(f"Detected texts in frame: {len(result)}")

    # Concatenate all detected texts in the current frame
    detected_texts = []
    for detection in result:
        text = detection[1]
        ocr_confidence = detection[2]  # Confidence of the OCR result
        detected_texts.append((text, ocr_confidence))  # Store text and its confidence

    # Check if any detected texts are present
    if detected_texts:
        # Combine texts into a single string, filtering out empty strings
        combined_text = ' '.join(text for text, _ in detected_texts if text.strip())

        # Calculate average OCR confidence
        avg_ocr_confidence = sum(conf for _, conf in detected_texts) / len(detected_texts)

        # Store the combined text, average OCR confidence, YOLO confidence, and the cropped frame
        ocr_results.append((combined_text, avg_ocr_confidence, yolo_confidence, cropped_frame))

# Sort results by EasyOCR confidence
ocr_results.sort(key=lambda x: x[1], reverse=True)  # Sort by EasyOCR confidence

# Display the results
for idx, (combined_text, easyocr_conf, yolo_conf, cropped_frame) in enumerate(ocr_results):
    print(f"Result {idx + 1}: Detected texts: '{combined_text}'")
    print(f" - Average EasyOCR Confidence: {easyocr_conf:.2f}")
    print(f" - YOLO Confidence: {yolo_conf:.2f}")

    # Show the cropped frame
    cv2_imshow(cropped_frame)
